In [13]:
import numpy as np
from PIL import Image
import brotli
import struct

def delta_encode_rgb(image_array):
    image_array = image_array.astype(np.int16)
    delta = np.zeros_like(image_array, dtype=np.uint8)
    delta[0] = image_array[0]
    delta[1:] = ((image_array[1:] - image_array[:-1]) % 256).astype(np.uint8)
    return delta

def bitpack_rgb(delta_array):
    return delta_array.astype(np.uint8).tobytes()  # No downshift!

def compress_rgb_with_hybrid(image_array):
    delta = delta_encode_rgb(image_array)
    packed = bitpack_rgb(delta)
    compressed = brotli.compress(packed)
    return compressed

def delta_encode_alpha(alpha_channel):
    alpha_channel = alpha_channel.astype(np.int16)
    delta = np.zeros_like(alpha_channel, dtype=np.uint8)
    delta[0] = alpha_channel[0]
    delta[1:] = ((alpha_channel[1:] - alpha_channel[:-1]) % 256).astype(np.uint8)
    return delta

def compress_alpha_channel(alpha_channel):
    flat_alpha = alpha_channel.flatten()
    delta_alpha = delta_encode_alpha(flat_alpha)
    compressed = brotli.compress(delta_alpha.tobytes())
    return compressed, b'\x01'

def is_opaque(alpha_channel):
    return np.all(alpha_channel == 255)

def is_strict_binary_alpha(alpha_channel):
    unique_vals = np.unique(alpha_channel)
    return np.array_equal(unique_vals, [0]) or np.array_equal(unique_vals, [0, 255])

def save_lix_rgba(image_path, output_path):
    img = Image.open(image_path).convert("RGBA")
    arr = np.array(img)
    h, w, _ = arr.shape

    rgb_data = arr[:, :, :3].reshape(-1, 3)
    alpha_data = arr[:, :, 3]

    rgb_compressed = compress_rgb_with_hybrid(rgb_data)

    if is_opaque(alpha_data):
        alpha_compressed = b''
        alpha_method = b'\x00'
    elif is_strict_binary_alpha(alpha_data):
        alpha_flat = alpha_data.flatten()
        alpha_compressed = brotli.compress(alpha_flat.tobytes())
        alpha_method = b'\x02'
    else:
        alpha_compressed, alpha_method = compress_alpha_channel(alpha_data)

    header = (
        b'LIX' +
        b'\x04' +
        b'\x05' +
        alpha_method +
        struct.pack('>HH', w, h) +
        struct.pack('>I', len(rgb_compressed)) +
        struct.pack('>I', len(alpha_compressed))
    )

    with open(output_path, "wb") as f:
        f.write(header + rgb_compressed + alpha_compressed)

    print(f"✅ Saved .lix RGBA file: {output_path}, total size: {len(header + rgb_compressed + alpha_compressed)} bytes")

# Call this to encode
save_lix_rgba("exp_rgba.png", "image_rgba_exp.lix")


✅ Saved .lix RGBA file: image_rgba_exp.lix, total size: 14726 bytes


In [14]:
import numpy as np
from PIL import Image
import brotli
import struct

def delta_decode_rgb(packed):
    unpacked = np.frombuffer(packed, dtype=np.uint8).astype(np.int16).reshape(-1, 3)

    result = np.zeros_like(unpacked, dtype=np.int16)
    result[0] = unpacked[0]

    for i in range(1, len(unpacked)):
        result[i] = (result[i - 1] + unpacked[i]) % 256

    return result.astype(np.uint8)

def delta_decode_alpha(data):
    arr = np.frombuffer(data, dtype=np.uint8).astype(np.int16)
    result = np.zeros_like(arr, dtype=np.int16)
    result[0] = arr[0]
    for i in range(1, len(arr)):
        result[i] = (result[i - 1] + arr[i]) % 256
    return result.astype(np.uint8)

def read_lix_rgba(path):
    with open(path, 'rb') as f:
        data = f.read()

    assert data[:3] == b'LIX', "Invalid magic bytes"
    mode = data[3]
    rgb_method = data[4]
    alpha_method = data[5]
    w, h = struct.unpack('>HH', data[6:10])
    rgb_size = struct.unpack('>I', data[10:14])[0]
    alpha_size = struct.unpack('>I', data[14:18])[0]

    rgb_compressed = data[18:18 + rgb_size]
    alpha_compressed = data[18 + rgb_size:18 + rgb_size + alpha_size]

    rgb_packed = brotli.decompress(rgb_compressed)
    rgb_array = delta_decode_rgb(rgb_packed).reshape(h, w, 3)

    if alpha_method == 0:
        alpha_channel = np.full((h * w,), 255, dtype=np.uint8)
    elif alpha_method == 1:
        decoded = brotli.decompress(alpha_compressed)
        alpha_flat = delta_decode_alpha(decoded)
        alpha_channel = alpha_flat
    elif alpha_method == 2:
        alpha_flat = brotli.decompress(alpha_compressed)
        alpha_channel = np.frombuffer(alpha_flat, dtype=np.uint8)
    else:
        raise ValueError(f"Unknown alpha method: {alpha_method}")

    alpha_channel = alpha_channel.reshape(h, w)

    rgba = np.dstack([rgb_array, alpha_channel])
    return Image.fromarray(rgba, mode="RGBA")

# Call this to decode
img = read_lix_rgba("image_rgba_exp.lix")
img.save("decoded_rgba_output.png")
print("✅ RGBA image decoded and saved as decoded_rgba_output.png")


✅ RGBA image decoded and saved as decoded_rgba_output.png
